# Projective Preferential Bayesian Optimization: DSGE prior elicitation

<font size="4">In this notebook the expert knowledge about DSGE priors can be elicited by using the [Projective Preferential Bayesian Optimization](https://arxiv.org/abs/2002.03113) framework. The test case is the New Keynesian sticky price model of Calvo (1983). </font> 

#### Import dependencies

In [1]:
%matplotlib widget

In [2]:
import warnings
warnings.filterwarnings(action='ignore')
import time
import sys
from datetime import datetime
import numpy as np
from gp_model import GPModel
from ppbo_settings import PPBO_settings
from acquisition import next_query
import DSGE.GUI as GUI
from jupyter_ui_poll import run_ui_poll_loop
from ipywidgets.widgets import HBox
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 45em; }</style>")) #Make outputwindow larger

#### Define the problem setting and the aquisition strategy
There are six possible acquisition startegies:
- PCD = preferential coordinate descent
- EI = expected improvement by projective preferential query
- EXT = pure exploitation
- EXR = pure exploration (variance maximization)
- RAND = random 

'user_feedback_grid_size' determines the granularity of the slider, that is, the number of options available for the user. Each option is run in a dedicated CPU. Hence, the optimal performance will be achieved if 'user_feedback_grid_size' is a multiple of the number of CPUs. 

In [3]:
PPBOset = PPBO_settings(D=7,bounds=((0.2,0.8),(0.2,0.8),(0.2,0.8),(0.2,0.8),(2,15),(0.25,3),(0.3,0.9)),
                    xi_acquisition_function='EI_fixed_x',user_feedback_grid_size=10,verbose=False)

#### Set initial queries

In [4]:
initial_queries_xi = np.diag([PPBOset.original_bounds[i][1] for i in range(PPBOset.D)])
initial_queries_x = np.random.uniform([PPBOset.original_bounds[i][0] for i in range(PPBOset.D)], [PPBOset.original_bounds[i][1] for i in range(PPBOset.D)], (len(initial_queries_xi), PPBOset.D))

#### Querying settings

In [5]:
NUMBER_OF_QUERIES = 10  
ADAPTIVE_INITIALIZATION = True  #In initilization: immediatly update the coordinate according to the user feedback

#### Hyperparameter optimization

In [6]:
OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_ACTUAL_QUERY_NUMBER = 1000 

#### Initialize the user session

In [7]:
should_log = False
if should_log:
    orig_stdout = sys.stdout
    log_file = open('DSGE/user_session_log_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.txt', "w")
    sys.stdout = log_file
GUI_ses = GUI.GUI_session(PPBOset)
results_mu_star = []
results_x_star = []
results_x_star_unscaled = []

## PPBO loops

In [8]:
start = time.time() 

### 1. Initialization loop

In [ ]:
for i in range(len(initial_queries_xi)):
    ''' Projective preferential query '''
    xi = initial_queries_xi[i].copy()
    if not i==0 and GUI_ses.user_feedback is not None and ADAPTIVE_INITIALIZATION:
        initial_queries_x[i:,:] = GUI_ses.user_feedback
    x = initial_queries_x[i].copy()
    x[xi!=0] = 0
    GUI_ses.set_x(x)
    GUI_ses.set_xi(xi)
    ''' Event loop '''
    button,slider,fig,l1,l2,l3,l4 = GUI_ses.prepare_app()
    app = HBox(children=(slider,button))
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            GUI_ses.update_plot(l1,l2,l3,l4,fig,slider)  
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    dt = run_ui_poll_loop(wait_user_input)
    ''' Create GP model for first time '''
    if i==0:
        GP_model = GPModel(PPBOset)
    ''' Update GP model '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.update_data()
    GP_model.update_model()
    ''' Save the predictive mean maximum and maximizer'''
    results_mu_star.append(GP_model.mu_star_)
    results_x_star.append(GP_model.x_star_)
    results_x_star_unscaled.append(GP_model.FP.unscale(GP_model.x_star_))
if OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION:
    GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_INITIALIZATION)  
print("Initialization done!")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2. Main loop

In [ ]:
for i in range(NUMBER_OF_QUERIES):
    #print("Starting query " + str(i+1)+"/"+str(NUMBER_OF_QUERIES)+" ...")
    ''' Next projective preferential query '''
    xi_next,x_next = next_query(PPBOset,GP_model,unscale=True)
    GUI_ses.set_xi(xi_next)
    GUI_ses.set_x(x_next)
    ''' Event loop '''
    button,slider,fig,l1,l2,l3,l4 = GUI_ses.prepare_app()
    app = HBox(children=(slider,button))
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            GUI_ses.update_plot(l1,l2,l3,l4,fig,slider)  
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    dt = run_ui_poll_loop(wait_user_input)
    ''' Append the user feedback '''
    GP_model.update_feedback_processing_object(np.array(GUI_ses.results))
    GP_model.mu_star_previous_iteration = GP_model.mu_star_
    ''' Update the model '''
    GP_model.update_data()
    if i+1==OPTIMIZE_HYPERPARAMETERS_AFTER_ACTUAL_QUERY_NUMBER:
        GP_model.update_model(optimize_theta=True)     
    else:
        GP_model.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION)
    ''' Save the predictive mean maximum and maximizer'''
    #print("x_star of the iteration: " + str(GP_model.x_star_))
    results_mu_star.append(GP_model.mu_star_)
    results_x_star.append(GP_model.x_star_)
    results_x_star_unscaled.append(GP_model.FP.unscale(GP_model.x_star_))
print("Elicitation done!")

In [ ]:
print("---------------------------------------")
print("The session completed!")
print("Total time: " + str(time.time()-start) + " seconds.")
x_star_, mu_star_  = GP_model.mu_star(mu_star_finding_trials=10)
x_star_unscaled = GP_model.FP.unscale(x_star_)
print("The final x_star: " + str(x_star_))

## Save the results

#### Save the user session results

In [ ]:
#Save results to csv-file
print("Saving the user session results...")
user_session_results = GUI_ses.results.copy()
user_session_results['iter_mu_star'] = results_mu_star
user_session_results['iter_x_star'] = results_x_star
user_session_results['iter_x_star_unscaled'] = results_x_star_unscaled
user_session_results.to_csv('DSGE/user_session_results_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
#Close the log-file
if should_log:
    sys.stdout = orig_stdout
    log_file.close()

## Analyze the results

#### The user's most preferred configuration

In [ ]:
print(str(x_star_unscaled))

#### Find parametric priors that are in KL-sense closest to the expert's opinion

from prior_elicitation import Prior, g_theta, minimize_KL, sample_h, plot
from scipy.stats import norm, beta
from matplotlib import pyplot as plt

lambda_sample = sample_h(GP_model,sample_size=100,n_input_points=1000)

fixed_hyperparams = [0.2,0.2,0.2,0.2,1.5,0.375,0.1] #Scale parameters of the priors
prior0 = Prior(family='beta',valrange=(0,1),lambda_indices=[0],fixed_hyperparams=[fixed_hyperparams[0]],is_location_param=True)
prior1 = Prior(family='beta',valrange=(0,1),lambda_indices=[1],fixed_hyperparams=[fixed_hyperparams[1]],is_location_param=True) 
prior2 = Prior(family='beta',valrange=(0,1),lambda_indices=[2],fixed_hyperparams=[fixed_hyperparams[2]],is_location_param=True) 
prior3 = Prior(family='beta',valrange=(0,1),lambda_indices=[3],fixed_hyperparams=[fixed_hyperparams[3]],is_location_param=True) 
prior4 = Prior(family='normal',valrange=(2,15),lambda_indices=[4],fixed_hyperparams=[fixed_hyperparams[4]],is_location_param=True) 
prior5 = Prior(family='normal',valrange=(0.25,3),lambda_indices=[5],fixed_hyperparams=[fixed_hyperparams[5]],is_location_param=True) 
prior6 = Prior(family='beta',valrange=(0,1),lambda_indices=[6],fixed_hyperparams=[fixed_hyperparams[6]],is_location_param=True) 

Prior0: Beta(x,0.2) for parameter $\rho_{Z}$

thetas = np.arange(prior0.domain[0], prior0.domain[1], 0.01)
g = [g_theta(theta,prior0,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior0)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior0.fixed_hyperparams[0]),g,thetas,fig_id=10)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.977, prior0.fixed_hyperparams[0]), c='green')

Prior1: Beta(x,0.2) for parameter $\rho_{g}$

thetas = np.arange(prior1.domain[0], prior1.domain[1], 0.005)
g = [g_theta(theta,prior1,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior1)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior1.fixed_hyperparams[0]),g,thetas,fig_id=11)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.957, prior1.fixed_hyperparams[0]), c='green')

Prior2: Beta(x,0.2) for parameter $\theta_{p}$

thetas = np.arange(prior2.domain[0], prior2.domain[1], 0.005)
g = [g_theta(theta,prior2,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior2)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior2.fixed_hyperparams[0]),g,thetas,fig_id=12)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.648, prior2.fixed_hyperparams[0]), c='green')

Prior3: Beta(x,0.2) for parameter $\theta_{w}$

thetas = np.arange(prior3.domain[0], prior3.domain[1], 0.005)
g = [g_theta(theta,prior3,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior3)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior3.fixed_hyperparams[0]),g,thetas,fig_id=13)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.649, prior3.fixed_hyperparams[0]), c='green')

Prior4: N(x,1.5) for parameter inv_adj_cost

thetas = np.arange(prior4.domain[0], prior4.domain[1], 0.01)
g = [g_theta(theta,prior4,lambda_sample) for theta in thetas]
q = norm.pdf
opt_hyperparam = minimize_KL(q,g,prior4)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior4.fixed_hyperparams[0]),g,thetas,fig_id=14)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 5.632, prior4.fixed_hyperparams[0]), c='green')

Prior5: N(x,0.375) for parameter $\sigma_c$

thetas = np.arange(prior5.domain[0], prior5.domain[1], 0.01)
g = [g_theta(theta,prior5,lambda_sample) for theta in thetas]
q = norm.pdf
opt_hyperparam = minimize_KL(q,g,prior5)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior5.fixed_hyperparams[0]),g,thetas,fig_id=15)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 1.267, prior5.fixed_hyperparams[0]), c='green')

Prior6: Beta(x,0.1) for parameter $h$

thetas = np.arange(prior6.domain[0], prior6.domain[1], 0.005)
g = [g_theta(theta,prior6,lambda_sample) for theta in thetas]
q = beta.pdf
opt_hyperparam = minimize_KL(q,g,prior6)

#Plot parametric and non-parametric priors elicited from the user
plot(q(thetas, opt_hyperparam, prior6.fixed_hyperparams[0]),g,thetas,fig_id=16)
#Plot true target (posterior estimated on the extended sample 1984-2015)
plt.plot(thetas, q(thetas, 0.504, prior6.fixed_hyperparams[0]), c='green')